In [1]:
from transformers import BertTokenizer, TFBertModel
from pymongo import MongoClient, ASCENDING, DESCENDING, HASHED
from tqdm.notebook import tqdm

conn = MongoClient()

fake_fibvid = conn.fake_fibvid
tweets  = fake_fibvid.tweets

process_fibvid = conn.process_fibvid
tweets_bert_tokens  = process_fibvid.tweets_bert_tokens

tweets_bert_tokens.create_index([('num_tokens', ASCENDING)])
tweets_bert_tokens.create_index([('created', ASCENDING)])
tweets_bert_tokens.create_index([('tweetId', HASHED)])
tweets_bert_tokens.create_index([('userId', HASHED)])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

D:\FibVid\FakeEnv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fanat\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
total = tweets.count_documents({})

to_store = []

for tweet in tqdm(tweets.find(), total=total):
    tokens = tokenizer(tweet['text'], truncation=True)
    tokens['num_tokens'] = len(tokens['input_ids'])
    tokens['created'] = tweet['created']
    tokens['userId'] = tweet['userId']
    tokens['tweetId'] = tweet['tweetId']
    to_store.append(tokens)
    if len(to_store) > 10000:
        tweets_bert_tokens.insert_many(to_store)
        to_store = []
        

if len(to_store) > 0:
    tweets_bert_tokens.insert_many(to_store)
    to_store = []

  0%|          | 0/299118 [00:00<?, ?it/s]